<a href="https://colab.research.google.com/github/laura314159265/poly/blob/main/PR%C3%80CTICA_GUIADA/M%C3%88TODES_COR_PROGRAMA_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Partim de la EDP:


$c_vρ\frac{∂T}{∂t}=∇⋅(κ{\vec\nabla T})+P_{ext}$

Després de normalitzar i reduïr els paràmetres al mínim obtenim:

$\frac{\partial\tilde{T}}{\partial\tilde{t}}=\frac{\partial^2\tilde{T}}{\partial\tilde{x}^2}+1$



Discretitzem via Euler explícit amb la derivada per la dreta:

$\frac{T_{i+1}^{(n)}-T_i^{(n)}}{Δt}=1+\frac{T_i^{(n+1)}-2T_i^{(n)}+T_i^{(n-1)}}{(Δx)^2}$

Aïllem $T_{i+1}^{(n)}$:

$T_{i+1}^{(n)}=Δt+\alpha T_{i}^{(n-1)}+βT_{i}^{(n)} + α T_{i}^{(n+1)}$

On: $\ α=\frac{Δt}{\Delta x^2},\ β=1-\frac{2\Delta t}{\Delta x^2}$

Això es pot escriure en forma matricial i calcular tots els punts espaials de la malla a temps $i+1$ a partir dels de i. Ja que tenim tots els punts de $i=0$ de les condicions de contorn

I les condicions de contorn són:

*   $T(0,x)=36,5\ ℃$
*   $\frac{\partial T(t,\ 0)}{\partial t}=\frac{\partial T(t,\ 2\ cm)}{\partial t}=0$



In [81]:
#EULER EXPLÍCIT

import numpy as np
from scipy.linalg import toeplitz
import matplotlib.pyplot as plt

#Dades de l'enunciat
C_v= 3686
rho= 1081
kappa= 0.56
sigma= 0.472
volts= 40

N= 101   # Mallat espaial

x_0= 0.02
x_f= 2/x_0
x_i= 0

alpha_var= kappa/(C_v*rho)
print("La difusivitat dona:", alpha_var)

t_i= 0
t_0= x_0**2/alpha_var
print("t_0 és:", t_0)
t_f= 0.025
print("t_f és:", t_f)

#Variacions de les variables (els tres delta t corresponen als 3 de l'enunciat)
dx= x_f/N
print("El dx és:", dx)
dt_1= 0.51*dx**2 # disc temps 1
print("dt_1 és:", dt_1)
dt_2= 0.49*dx**2 # disc temps 2
dt_3= 0.25*dx**2  # disc temps 3

alpha= dt_1/(dx**2)
beta= 1-2*dt_1/(dx**2)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#Creació de la matriu
col1= [0]*(N-1)
col1[0]= beta
col1[1]= alpha
fila1= [0]*(N-1)
fila1[1]= alpha

matriu= toeplitz(col1, fila1) #Funció que crea una matriu diagonal per blocs a
                              #partir de la primera columna i la primera fila
print(matriu)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
P_ext= volts**2*sigma/(x_f*0.01)**2

T_0= P_ext/(C_v*rho)*t_0
T_init= 36.5/T_0   #Temperatura inicial
print("T_init:",T_init)
T_i= np.ones((N,1))*T_init  #Vector columna de les T_i per cada n (punt de l'espai)
deltat= np.ones((N,1))*dt_1 #Vector columna de Δt per sumar a l'expressió


La difusivitat dona: 1.405422823966274e-07
t_0 és: 2846.1185714285716
t_f és: 0.025
El dx és: 0.9900990099009901
dt_1 és: 0.4999509851975296
[[-0.02  0.51  0.   ...  0.    0.    0.  ]
 [ 0.51 -0.02  0.51 ...  0.    0.    0.  ]
 [ 0.    0.51 -0.02 ...  0.    0.    0.  ]
 ...
 [ 0.    0.    0.   ... -0.02  0.51  0.  ]
 [ 0.    0.    0.   ...  0.51 -0.02  0.51]
 [ 0.    0.    0.   ...  0.    0.51 -0.02]]
T_init: 67.66419491525423


Bloc del càlcul amb Euler explícit:

In [63]:
#EULER CÀLCUL
def Euler_explícit (T_i, t_f, dt, dx):
  i=1
  t_i=0
  x_i=0
  while t_i<t_f:

    T_i= np.matmul(matriu,T_i)++deltat
    t_i= t_i + dt
    x_i= x_i + dx

    #print(t_i)
    i=i+1
  print(f"El resultat de la iteració {i} és", T_i*T_0)
  """
  inputs: T_i (array de les temperatures inicials),
  t_f (temps final),
  dt (variació de temps),
  dx (varticaió d'espai)
  """

Euler_explícit(T_i, t_f, dt_1, dx)
Euler_explícit(T_i, t_f, dt_2, dx)
Euler_explícit(T_i, t_f, dt_3, dx)

El resultat de la iteració 502 és [[87990.4157009 ]
 [48982.11246697]
 [85138.67907531]
 [49078.01779598]
 [82402.53625186]
 [49062.86241312]
 [79773.09881824]
 [48945.18616554]
 [77242.16182329]
 [48732.87224158]
 [74802.15122327]
 [48433.19766366]
 [72446.07536923]
 [48052.87989845]
 [70167.48022454]
 [47598.11988305]
 [67960.40802611]
 [47074.64174283]
 [65819.35912407]
 [46487.72945509]
 [63739.25675593]
 [45842.26069361]
 [61715.41452923]
 [45142.73807075]
 [59743.50640464]
 [44393.31797723]
 [57819.53898715]
 [43597.83720434]
 [55939.82594788]
 [42759.83751905]
 [54100.96441268]
 [41882.58834944]
 [52299.81316634]
 [40969.10772584]
 [50533.47253265]
 [40022.18161165]
 [48799.26580168]
 [39044.38174779]
 [47094.72208511]
 [38038.08212499]
 [45417.56048987]
 [37005.47418955]
 [43765.67550873]
 [35948.58087977]
 [42137.12353432]
 [34869.26958314]
 [40530.11041001]
 [33769.26409719]
 [38942.97993819]
 [32650.15567062]
 [37374.20327207]
 [31513.41319543]
 [35822.36912323]
 [30360.3926